#### Packages

In [1]:
import pandas as pd
import numpy as np
import time
# import spacy # pour la lemmatisation
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier

On commence par charger les trois datasets que nous allons utiliser et on uniformise la labellisation avec une colonne "label"qui vaut 1 s'il s'agit d'une fake news et 0 sinon. On choisit de ne considérer que trois des datasets de l'article car le dernier,de taille beaucoup plus faible, ne permet pas dans l'article de Hoy et Koulouri d'obtenir des résultats corrects même sur une partie nouvelle du dataset.

## Chargement des bases

#### Chargement et mise en forme de ISOT

In [2]:
# Mise en forme dataset ISOT https://www.kaggle.com/datasets/csmalarkodi/isot-fake-news-dataset/
Isot_true_df = pd.read_csv("data/True.csv")
Isot_fake_df = pd.read_csv("data/Fake.csv")

#Création d'un dataset unique

Isot_true_df["label"] = 0  # Vraie news
Isot_fake_df["label"] = 1  # Fake news

Isot_data = pd.concat([Isot_true_df, Isot_fake_df], ignore_index=True)

Isot_data.head(5)

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [3]:
# On test la présence de NA
Isot_data.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [4]:
# On ne garde que le label et le text de l'article
Isot = Isot_data[['text', 'label']]

#### Chargement et mise en forme de fake_news

In [9]:
# Mise en forme dataset Fake_News https://www.kaggle.com/competitions/fake-news/data?select=train.csv
fake_news_data = pd.read_csv("data/train.csv")
fake_news_data.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
# On teste la présence de valeurs manquantes sur le texte, et on drop s'il y en a
fake_news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [11]:
fake_news_data = fake_news_data.dropna(subset=['text'])

In [12]:
# On ne garde que le label et le text de l'article et on lemmatise
fake_news = fake_news_data[['text', 'label']]
# start_time = time.time()
# fake_news['lemmatized_text'] = fake_news['text'].apply(lemmatize_text)
# print(f"Temps avec -1 processus : {time.time() - start_time} secondes")
fake_news.head(3)

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1


#### Chargement et mise en forme de fake_real

In [13]:
# Mise en forme dataset Fake_real https://www.kaggle.com/datasets/jillanisofttech/fake-or-real-news
fake_real_data = pd.read_csv("data/fake_or_real_news.csv")
fake_real_data['label'] = fake_real_data['label'].map({'FAKE': 1, 'REAL': 0})
fake_real_data.head(5)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


In [14]:
# On test la présence de NA
fake_real_data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [15]:
# On ne garde que le label et le text de l'article et on lemmatize
fake_real = fake_real_data[['text', 'label']]
# start_time = time.time()
# fake_real['lemmatized_text'] = fake_real['text'].apply(lemmatize_text)
# print(f"Temps avec -1 processus : {time.time() - start_time} secondes")
fake_real.head(3)

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,U.S. Secretary of State John F. Kerry said Mon...,0


A partir de la lecture de l'article référence, il apparait que sur ce jeu de données il est possible d'atteindre de très bonnes performances avec plusieurs features extraction et modèle de machine learning. On s'appuie donc pour commencer sur une approche semblable au V.E, qui repose sur l'utilisation d'un TF-IDF feature extracteur et une régression logistique (dans le but de relever ensuite les mots utilisés, à l'image de ce qu'ils font dans l'article).

A noter que l'on pourra ensuite imaginer d'autres approches, ou voir si l'on arrive à faire quelque chose de correct avec juste le titre etc...

# Construction d'un modèle "baseline" TF-IDF et régression logistique sur ISOT

Comme on est dans une situation où l'on ne cherche pas à ajuster des hyperparamètres, et que l'on sait que nous avons beaucoup de données donc on fait sans.

### Réalisation du modèle

In [13]:
# On réalise le split des données

X = Isot['text']  # Les articles/textes
y = Isot['label']  # Les labels (1 = Fake)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Vectorisation tfidf (ici probablement des paramètres à fine-tuned, notamment le max_df)

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [15]:
#On applique une régression logistique
model_tfidf_logistic = LogisticRegression()
model_tfidf_logistic.fit(X_train_tfidf, y_train)

LogisticRegression()

In [16]:
# Et on évalue notre petit modèle
y_pred = model_tfidf_logistic.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9863028953229399

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      4330
           1       0.99      0.98      0.99      4650

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980


Confusion Matrix:
 [[4284   46]
 [  77 4573]]


On trouve un modèle qui est donc déjà **très** efficace, mais ceci est cohérent avec l'article de Hoy et Koulouri, qui donnait déjà de très bon résultats avec ce genre de modèle.

On peut essayer de tester sur un nouveau dataset

### Test sur les autres datasets

In [19]:
# Définition d'une fonction de prédiction 

def apply_model_tfidf_logistic(new_data, text_column='text'):
    """
    Applique le modèle de détection de fake news entrainé sur ISOT et fonctionnant avec un tokenizer TF-IDF et une régression logistique à un nouveau DataFrame.
    Ajoute une colonne 'prediction' (0 = Real, 1 = Fake).
    """
    # Transformer les textes avec le TF-IDF déjà entraîné
    X_new_tfidf = vectorizer.transform(new_data[text_column])
    
    # Faire les prédictions
    predictions = model_tfidf_logistic.predict(X_new_tfidf)
    
    # Ajouter la colonne 'prediction'
    new_data = new_data.copy()  # Pour ne pas modifier le DataFrame original
    new_data['prediction'] = predictions
    
    return new_data
    

In [18]:
# Appliquer le modèle à Fake_News
fake_news_pred = apply_model_tfidf_logistic(fake_news)

print("\nÉvaluation sur Fake News :\n")
print("Accuracy:", accuracy_score(fake_news_pred['label'], fake_news_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_news_pred['label'], fake_news_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_news_pred['label'], fake_news_pred['prediction']))


Évaluation sur Fake News :

Accuracy: 0.5496845045999711

Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.18      0.29     10387
           1       0.53      0.92      0.67     10374

    accuracy                           0.55     20761
   macro avg       0.61      0.55      0.48     20761
weighted avg       0.61      0.55      0.48     20761


Confusion Matrix:
 [[1896 8491]
 [ 858 9516]]


In [19]:
# Appliquer le modèle à Fake_Real
fake_real_pred = apply_model_tfidf_logistic(fake_real)

print("\nÉvaluation sur Fake Real :\n")
print("Accuracy:", accuracy_score(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_real_pred['label'], fake_real_pred['prediction']))


Évaluation sur Fake News :

Accuracy: 0.5682715074980268

Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.23      0.35      3171
           1       0.54      0.91      0.68      3164

    accuracy                           0.57      6335
   macro avg       0.63      0.57      0.51      6335
weighted avg       0.63      0.57      0.51      6335


Confusion Matrix:
 [[ 727 2444]
 [ 291 2873]]


On a donc ici globalement un accuracy très faible, à peine supérieure à 50 % (qui est le random guess face à des datasets équilibrés...).
De façon intéressante, on a surtout un très faible recall des articles qui ne sont pas des fake-news. Autrement dit, un quart seulement des articles "vrais" sont reconnus comme tel. Notre modèle considère la plupart des articles qu'on lui présente comme des faux articles. Ainsi, il semble que le modèle ait appri particulièrement à reconnaître les vrais articles de notre dataset spécifiquement, mais qu'il a beaucoup de mal à reconnaître un vrai article venant d'un autre dataset.

### Modifier notre modèle pour améliorer le recall sur les autres datasets en ignorant Reuters

L'article de Hoy et Koulouri semble pointer vers le fait que cela peut-être du au fait que celui-ci contient beaucoup d'article de Reuters étiquetés comme vrais, ce qui n'est pas forcément le cas des autres datasets. On cherche donc à tirer profit de cette information pour améliorer le recall.

In [20]:
# On compte la présence de "reuters" (sans la casse) dans True.csv
reuters_count_true = Isot_true_df['text'].str.lower().str.count(r"\breuters\b").sum()
print(f'Le mot "Reuters" apparaît {reuters_count_true} fois dans True.csv.')

Le mot "Reuters" apparaît 28976 fois dans True.csv.


In [21]:
# On compte la présence de "reuters" (sans la casse) dans Fake.csv
reuters_count_fake = Isot_fake_df['text'].str.lower().str.count(r"\breuters\b").sum()
print(f'Le mot "Reuters" apparaît {reuters_count_fake} fois dans Fake.csv.')

Le mot "Reuters" apparaît 449 fois dans Fake.csv.


Clairement, ce mot apparait comme extrêmement discriminant entre les deux datasets, donc on se propose de l'ajouter à la stoplist pour voir si on a une amélioration (car il s'agit d'un biais de construction du dataset).

In [22]:
# On construit une nouvelle liste de stop_word ajoutant reuters

# Récupérer la liste de stop words en anglais de sklearn
default_stop_words = text.ENGLISH_STOP_WORDS

# Ajouter "reuters" à cette liste
custom_stop_words = list(default_stop_words.union({"reuters"}))

In [23]:
# Et on recommence la vectorisation
vectorizer_custom = TfidfVectorizer(stop_words=custom_stop_words, max_df=0.7)

X_train_tfidf_custom = vectorizer_custom.fit_transform(X_train)
X_test_tfidf_custom = vectorizer_custom.transform(X_test)

In [24]:
# On applique une régression logistique
model_tfidf_logistic_custom = LogisticRegression()
model_tfidf_logistic_custom.fit(X_train_tfidf_custom, y_train)

LogisticRegression()

In [25]:
# Et on évalue notre petit modèle
y_pred_custom = model_tfidf_logistic_custom.predict(X_test_tfidf_custom)

print("Accuracy:", accuracy_score(y_test, y_pred_custom))
print("\nClassification Report:\n", classification_report(y_test, y_pred_custom))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_custom))

Accuracy: 0.9776169265033408

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98      4330
           1       0.98      0.98      0.98      4650

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980


Confusion Matrix:
 [[4242   88]
 [ 113 4537]]


On remarque donc une baisse de accuracy de près de 1 point, mais ce n'est pas surprenant car on lui demande d'ignorer un mot très discriminant. Regardons surtout ce que cela donne sur les autres datasets.

In [26]:
# Définition d'une nouvelle fonction de prédiction 

def apply_model_tfidf_logistic_custom(new_data, text_column='text'):
    """
    Applique le modèle de détection de fake news entrainé sur ISOT et fonctionnant avec un tokenizer TF-IDF et une régression logistique à un nouveau DataFrame.
    Ajoute une colonne 'prediction' (0 = Real, 1 = Fake).
    """
    # Transformer les textes avec le TF-IDF déjà entraîné
    X_new_tfidf = vectorizer_custom.transform(new_data[text_column])
    
    # Faire les prédictions
    predictions = model_tfidf_logistic_custom.predict(X_new_tfidf)
    
    # Ajouter la colonne 'prediction'
    new_data = new_data.copy()  # Pour ne pas modifier le DataFrame original
    new_data['prediction'] = predictions
    
    return new_data

In [27]:
# Appliquer le nouveau modèle à Fake_News
fake_news_pred = apply_model_tfidf_logistic_custom(fake_news)

print("\nÉvaluation sur Fake News :\n")
print("Accuracy:", accuracy_score(fake_news_pred['label'], fake_news_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_news_pred['label'], fake_news_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_news_pred['label'], fake_news_pred['prediction']))


Évaluation sur Fake News :

Accuracy: 0.5592216174558066

Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.23      0.34     10387
           1       0.54      0.89      0.67     10374

    accuracy                           0.56     20761
   macro avg       0.61      0.56      0.51     20761
weighted avg       0.61      0.56      0.51     20761


Confusion Matrix:
 [[2375 8012]
 [1139 9235]]


In [28]:
# Appliquer le modèle à Fake_Real
fake_real_pred = apply_model_tfidf_logistic_custom(fake_real)

print("\nÉvaluation sur Fake News :\n")
print("Accuracy:", accuracy_score(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_real_pred['label'], fake_real_pred['prediction']))


Évaluation sur Fake News :

Accuracy: 0.5861089187056038

Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.30      0.42      3171
           1       0.55      0.88      0.68      3164

    accuracy                           0.59      6335
   macro avg       0.63      0.59      0.55      6335
weighted avg       0.63      0.59      0.55      6335


Confusion Matrix:
 [[ 943 2228]
 [ 394 2770]]


On remarque une amélioration de l'accuracy de 1 point environ pour fake_news, et de 2 points environ pour fake_real. C'est donc relativement peu. Mais le rappel des vrais article augmente assez nettement (compte tenu du fait que l'on a seulement ajouté un stop word), de 5 points pour le dataset fake_news et de 7 pour le fake_real, ce qui représente une très forte hausse, même si on garde des rappels particulièrement faibles.

Ceci permet donc de valider notrehypothèse selon laquelle le prédicteur que l'on construit semble particulièrement apprendre les caractéristiques spécifiques de notre dataset, et plus particulièrement des vrais articles.

#### Recherche d'autres mots à ignorer

Nous essayons donc de voir si d'autres mots peuvent apparaitre comme fortement discriminant mais liés à notre dataset spécifique.

In [29]:
# On crée un vectorizer spécifique à chaque dataset
vectorizer_true = TfidfVectorizer(stop_words='english', max_df=0.7)
vectorizer_fake = TfidfVectorizer(stop_words='english', max_df=0.7)

isot_true = Isot_true_df['text']
isot_fake = Isot_fake_df['text']

# Vectorisation sur les textes réels et fake
X_true_tfidf = vectorizer_true.fit_transform(isot_true)
X_fake_tfidf = vectorizer_fake.fit_transform(isot_fake)

In [30]:
# On crée une fonction pour retrouver les mots les plus marquant
def get_top_tfidf_words(X_tfidf, vectorizer, top_n=20):
    feature_names = vectorizer.get_feature_names_out()  # Récupère les mots du vocabulaire
    tfidf_means = np.asarray(X_tfidf.mean(axis=0)).flatten()  # Calcul de la moyenne des scores TF-IDF
    tfidf_scores = pd.DataFrame({
        'word': feature_names,
        'mean_tfidf': tfidf_means
    })
    return tfidf_scores.sort_values(by='mean_tfidf', ascending=False).head(top_n)

In [31]:
# Top mots pour les vraies news
top_true_words = get_top_tfidf_words(X_true_tfidf, vectorizer_true)

# Top mots pour les fausses news
top_fake_words = get_top_tfidf_words(X_fake_tfidf, vectorizer_fake)

print("Top mots des vraies news :")
print(top_true_words)

print("\nTop mots des fausses news :")
print(top_fake_words)

Top mots des vraies news :
             word  mean_tfidf
60979       trump    0.046345
46881   president    0.023024
28425       house    0.019371
56726       state    0.018735
50201  republican    0.017210
25505  government    0.016776
62152      united    0.015964
56741      states    0.015831
64116  washington    0.015222
41262         new    0.014864
42070       north    0.014215
44453       party    0.013760
44983      people    0.013741
60112        told    0.013433
13239     clinton    0.013385
19891    election    0.013228
53491      senate    0.013190
33496       korea    0.013153
51795      russia    0.013083
64631       white    0.013018

Top mots des fausses news :
            word  mean_tfidf
85390      trump    0.056855
73472       said    0.021114
66405  president    0.020861
20295    clinton    0.019777
60433      obama    0.018388
63823     people    0.018239
40460    hillary    0.016386
27832     donald    0.015360
46829       just    0.014459
50810       like    0.01

Et on ne trouve rien d'intéressant...

Pour le principe on regarde si cela donne quelquechose en prenant notre vectorizer général

In [32]:
# Vectorisation sur les textes réels et fake
X_true_tfidf = vectorizer.transform(Isot_true_df['text'])
X_fake_tfidf = vectorizer.transform(Isot_fake_df['text'])

top_true_words = get_top_tfidf_words(X_true_tfidf, vectorizer)
top_fake_words = get_top_tfidf_words(X_fake_tfidf, vectorizer)

print("Top mots des vraies news :")
print(top_true_words)

print("\nTop mots des fake news :")
print(top_fake_words)

Top mots des vraies news :
              word  mean_tfidf
101008       trump    0.043088
84300      reuters    0.029872
78617    president    0.024478
94017        state    0.020250
48040        house    0.020170
43308   government    0.018729
106531  washington    0.018109
83773   republican    0.018035
103228      united    0.017084
94048       states    0.016648
70730        north    0.015132
69361          new    0.015130
99617         told    0.014701
89019       senate    0.014301
56682        korea    0.014187
74712        party    0.014149
65420     minister    0.014081
22692        china    0.014012
34337     election    0.013818
86328       russia    0.013485

Top mots des fake news :
             word  mean_tfidf
101008      trump    0.058400
23720     clinton    0.021457
71560       obama    0.019400
75570      people    0.019103
78617   president    0.018740
46990     hillary    0.017905
54231        just    0.017221
59500        like    0.015496
32173      donald    0.014

Bon on ne trouve rien de mieu, donc laissons cette piste pour le moment...

### Faisons un test de baseline TF-IDF logistique sur Fake News

In [33]:
# On réalise le split des données

X = fake_news['text']  # Les articles/textes
y = fake_news['label']  # Les labels (1 = Fake)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# Vectorisation tfidf (ici probablement des paramètres à fine-tuned, notamment le max_df)

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [35]:
#On applique une régression logistique
model_tfidf_logistic = LogisticRegression()
model_tfidf_logistic.fit(X_train_tfidf, y_train)

LogisticRegression()

In [36]:
# Et on évalue notre petit modèle
y_pred = model_tfidf_logistic.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9501565133638333

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.94      0.95      2079
           1       0.94      0.96      0.95      2074

    accuracy                           0.95      4153
   macro avg       0.95      0.95      0.95      4153
weighted avg       0.95      0.95      0.95      4153


Confusion Matrix:
 [[1951  128]
 [  79 1995]]


In [37]:
# Appliquer le modèle à Fake_News
isot_pred = apply_model_tfidf_logistic(Isot)

print("\nÉvaluation sur Fake News :\n")
print("Accuracy:", accuracy_score(isot_pred['label'], isot_pred['prediction']))
print("\nClassification Report:\n", classification_report(isot_pred['label'], isot_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(isot_pred['label'], isot_pred['prediction']))


Évaluation sur Fake News :

Accuracy: 0.7342197870729208

Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.77      0.73     21417
           1       0.77      0.71      0.74     23481

    accuracy                           0.73     44898
   macro avg       0.74      0.74      0.73     44898
weighted avg       0.74      0.73      0.73     44898


Confusion Matrix:
 [[16407  5010]
 [ 6923 16558]]


In [38]:
fake_real_pred = apply_model_tfidf_logistic(fake_real)

print("\nÉvaluation sur Fake News :\n")
print("Accuracy:", accuracy_score(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_real_pred['label'], fake_real_pred['prediction']))


Évaluation sur Fake News :

Accuracy: 0.7758484609313339

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.58      0.72      3171
           1       0.70      0.97      0.81      3164

    accuracy                           0.78      6335
   macro avg       0.82      0.78      0.77      6335
weighted avg       0.82      0.78      0.77      6335


Confusion Matrix:
 [[1853 1318]
 [ 102 3062]]


Il ne semble pas y avoir le même drop sur le recallque pour les modèles entrainés sur ISOT, même si un peu sur ISOT justement. Et du coup on a de biens meilleures accuracy totales.

### Modèle baseline TF-IDF logistique sur Fake Real

In [39]:
# On réalise le split des données

X = fake_real['text']  # Les articles/textes
y = fake_real['label']  # Les labels (1 = Fake)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
# Vectorisation tfidf (ici probablement des paramètres à fine-tuned, notamment le max_df)

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [41]:
#On applique une régression logistique
model_tfidf_logistic = LogisticRegression()
model_tfidf_logistic.fit(X_train_tfidf, y_train)

LogisticRegression()

In [42]:
# Et on évalue notre petit modèle
y_pred = model_tfidf_logistic.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9155485398579322

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.90      0.91       639
           1       0.90      0.93      0.92       628

    accuracy                           0.92      1267
   macro avg       0.92      0.92      0.92      1267
weighted avg       0.92      0.92      0.92      1267


Confusion Matrix:
 [[573  66]
 [ 41 587]]


In [43]:
# Appliquer le modèle à Fake_News
fake_news_pred = apply_model_tfidf_logistic(fake_news)

print("\nÉvaluation sur Fake News :\n")
print("Accuracy:", accuracy_score(fake_news_pred['label'], fake_news_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_news_pred['label'], fake_news_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_news_pred['label'], fake_news_pred['prediction']))


Évaluation sur Fake News :

Accuracy: 0.7326236693800877

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.51      0.66     10387
           1       0.66      0.95      0.78     10374

    accuracy                           0.73     20761
   macro avg       0.79      0.73      0.72     20761
weighted avg       0.79      0.73      0.72     20761


Confusion Matrix:
 [[5341 5046]
 [ 505 9869]]


In [44]:
isot_pred = apply_model_tfidf_logistic(Isot)

print("\nÉvaluation sur ISOT :\n")
print("Accuracy:", accuracy_score(isot_pred['label'], isot_pred['prediction']))
print("\nClassification Report:\n", classification_report(isot_pred['label'], isot_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(isot_pred['label'], isot_pred['prediction']))


Évaluation sur ISOT :

Accuracy: 0.6824357432402334

Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.65      0.66     21417
           1       0.69      0.71      0.70     23481

    accuracy                           0.68     44898
   macro avg       0.68      0.68      0.68     44898
weighted avg       0.68      0.68      0.68     44898


Confusion Matrix:
 [[13973  7444]
 [ 6814 16667]]


On retrouve des choses assez semblables avec ce que l'on a vu sur Fake News. Là non plus, on n'a pas une chute de recall sur les "vraies" nouvelles.

To do : tester un autre modèle que la regression logistique ?

Tester un autre feature ? --> aller sur Bert.

Tester un mix des datasets ? Les deux plus gros ?

# Test sur le panachage de datasets

#### True de Fake News et False de ISOT

Ici, on part de l'idée que notre modèle sur-apprend à reconnaitre les vraies nouvelles de ISOT, donc on test ce que cela donne si on crée un nouveau dtaset à partir des Fake News de ISOT et des vraies de Fake News (on choisit de se baser sur ces deux datasets car ils ont des tailles similaires)

In [18]:
# On crée un new_data avec les vraies de Fake_news et les fausses de ISOT (et le vomplémentaire new_data_2)

# Comme ISOT est deux fois plus grand, on ne prendsque 50 % des observations de ISOT
isot_sub = Isot.sample(frac=0.5, random_state=42)

# Création new_data
new_data = pd.concat([
    isot_sub[isot_sub['label'] == 1], 
    fake_news[fake_news['label'] == 0]
], ignore_index=True)

print(new_data['label'].value_counts())
print(new_data.head())

# Création new_data_2
new_data_2 = pd.concat([
    isot_sub[isot_sub['label'] == 0], 
    fake_news[fake_news['label'] == 1]
], ignore_index=True)

print(new_data_2['label'].value_counts())
print(new_data_2.head())

label
1    11784
0    10387
Name: count, dtype: int64
                                                text  label
0  Donald Trump s White House is in chaos, and th...      1
1  Now that Donald Trump is the presumptive GOP n...      1
2  Mike Pence is a huge homophobe. He supports ex...      1
3  Twisted reasoning is all that comes from Pelos...      1
4   The goal of socialism is communism.  -Vladimi...      1
label
0    10665
1    10374
Name: count, dtype: int64
                                                text  label
0  SAN FRANCISCO (Reuters) - California Attorney ...      0
1  WASHINGTON (Reuters) - As a lawyer in private ...      0
2  ADEN (Reuters) - A Salafist imam was shot dead...      0
3  KUALA LUMPUR (Reuters) - Potential witnesses t...      0
4  DUBAI (Reuters) - Eight women and two children...      0


In [20]:
# Notre modèle sur new_data

# On réalise le split des données

X = new_data['text']  # Les articles/textes
y = new_data['label']  # Les labels (1 = Fake)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorisation tfidf (ici probablement des paramètres à fine-tuned, notamment le max_df)

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

#On applique une régression logistique
model_tfidf_logistic = LogisticRegression()
model_tfidf_logistic.fit(X_train_tfidf, y_train)

# Et on évalue notre petit modèle
y_pred = model_tfidf_logistic.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.943630214205186

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      2065
           1       0.94      0.95      0.95      2370

    accuracy                           0.94      4435
   macro avg       0.94      0.94      0.94      4435
weighted avg       0.94      0.94      0.94      4435


Confusion Matrix:
 [[1925  140]
 [ 110 2260]]


In [21]:
# Appliquer le modèle à fake_real
fake_real_pred = apply_model_tfidf_logistic(fake_real)

print("\nÉvaluation sur Fake real :\n")
print("Accuracy:", accuracy_score(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_real_pred['label'], fake_real_pred['prediction']))


Évaluation sur Fake real :

Accuracy: 0.5294396211523283

Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.29      0.38      3171
           1       0.52      0.77      0.62      3164

    accuracy                           0.53      6335
   macro avg       0.54      0.53      0.50      6335
weighted avg       0.54      0.53      0.50      6335


Confusion Matrix:
 [[ 921 2250]
 [ 731 2433]]


In [22]:
# Appliquer le modèle à new_data_2
new_data_2_pred = apply_model_tfidf_logistic(new_data_2)

print("\nÉvaluation sur new_data_2_pred :\n")
print("Accuracy:", accuracy_score(new_data_2_pred['label'], new_data_2_pred['prediction']))
print("\nClassification Report:\n", classification_report(new_data_2_pred['label'], new_data_2_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(new_data_2_pred['label'], new_data_2_pred['prediction']))


Évaluation sur new_data_2_pred :

Accuracy: 0.7341603688388232

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.68      0.72     10665
           1       0.71      0.79      0.75     10374

    accuracy                           0.73     21039
   macro avg       0.74      0.73      0.73     21039
weighted avg       0.74      0.73      0.73     21039


Confusion Matrix:
 [[7241 3424]
 [2169 8205]]


On a donc finalement très peu de gain, et surtout la même tendance à sous-reconnaitre les vraies articles. Cela semble indiquer que le problème n'est pa forcément lié uniquement au "vraies" nouvelles de ISOT mais aussi à ses fausses nouvelles. Cela nous encourage donc à regarder ce qu'il se passe lorsque l'on effectue la même avec new_data_2

In [23]:
# Notre modèle sur new_data

# On réalise le split des données

X = new_data_2['text']  # Les articles/textes
y = new_data_2['label']  # Les labels (1 = Fake)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorisation tfidf (ici probablement des paramètres à fine-tuned, notamment le max_df)

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

#On applique une régression logistique
model_tfidf_logistic = LogisticRegression()
model_tfidf_logistic.fit(X_train_tfidf, y_train)

# Et on évalue notre petit modèle
y_pred = model_tfidf_logistic.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9812262357414449

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98      2190
           1       0.98      0.98      0.98      2018

    accuracy                           0.98      4208
   macro avg       0.98      0.98      0.98      4208
weighted avg       0.98      0.98      0.98      4208


Confusion Matrix:
 [[2159   31]
 [  48 1970]]


In [24]:
# Appliquer le modèle à fake_real
fake_real_pred = apply_model_tfidf_logistic(fake_real)

print("\nÉvaluation sur Fake real :\n")
print("Accuracy:", accuracy_score(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_real_pred['label'], fake_real_pred['prediction']))


Évaluation sur Fake real :

Accuracy: 0.7108129439621153

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.44      0.61      3171
           1       0.64      0.98      0.77      3164

    accuracy                           0.71      6335
   macro avg       0.80      0.71      0.69      6335
weighted avg       0.80      0.71      0.69      6335


Confusion Matrix:
 [[1404 1767]
 [  65 3099]]


In [25]:
# Appliquer le modèle à new_data
new_data_pred = apply_model_tfidf_logistic(new_data)

print("\nÉvaluation sur new_data_2_pred :\n")
print("Accuracy:", accuracy_score(new_data_pred['label'], new_data_pred['prediction']))
print("\nClassification Report:\n", classification_report(new_data_pred['label'], new_data_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(new_data_pred['label'], new_data_pred['prediction']))


Évaluation sur new_data_2_pred :

Accuracy: 0.6485499075368725

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.35      0.48     10387
           1       0.61      0.91      0.73     11784

    accuracy                           0.65     22171
   macro avg       0.70      0.63      0.61     22171
weighted avg       0.69      0.65      0.62     22171


Confusion Matrix:
 [[ 3599  6788]
 [ 1004 10780]]


Très bizarre, on obtient un modèle qui est _meilleur_ donc il semble que en réalité ce soit plutôt les Fake News de ISOT qui soit très spécifiques...

Testons donc un dataset mélangé

In [27]:
# Création de dataset mélangé

# Séparer les vraies et fausses news
isot_real = isot_sub[isot_sub['label'] == 1]
isot_fake = isot_sub[isot_sub['label'] == 0]

fake_news_real = fake_news[fake_news['label'] == 1]
fake_news_fake = fake_news[fake_news['label'] == 0]

# On sépare aléatoirement nos datasets
isot_real_1 = isot_real.sample(frac=0.5, random_state=42)
isot_real_2 = isot_real.drop(isot_real_1.index)

isot_fake_1 = isot_fake.sample(frac=0.5, random_state=42)
isot_fake_2 = isot_fake.drop(isot_fake_1.index)

fake_news_real_1 = fake_news_real.sample(frac=0.5, random_state=42)
fake_news_real_2 = fake_news_real.drop(fake_news_real_1.index)

fake_news_fake_1 = fake_news_fake.sample(frac=0.5, random_state=42)
fake_news_fake_2 = fake_news_fake.drop(fake_news_fake_1.index)

# Créer mixed_data_1 avec 50 % des vraies et fausses informations de chaque dataset
mixed_data_1 = pd.concat([isot_real_1, isot_fake_1, fake_news_real_1, fake_news_fake_1], ignore_index=True)

# Créer mixed_data_2 avec le reste des observations non présentes dans mixed_data_1
mixed_data_2 = pd.concat([isot_real_2, isot_fake_2, fake_news_real_2, fake_news_fake_2], ignore_index=True)

# Mélanger les datasets pour éviter toute structure dans les données
mixed_data_1 = mixed_data_1.sample(frac=1, random_state=5).reset_index(drop=True)
mixed_data_2 = mixed_data_2.sample(frac=1, random_state=6).reset_index(drop=True)

# Vérification
print("Mixed Data 1:")
print(mixed_data_1['label'].value_counts())

print("\nMixed Data 2:")
print(mixed_data_2['label'].value_counts())

Mixed Data 1:
label
1    11079
0    10526
Name: count, dtype: int64

Mixed Data 2:
label
1    11079
0    10526
Name: count, dtype: int64


In [28]:
# Notre modèle sur mixed_data_1

# On réalise le split des données

X = mixed_data_1['text']  # Les articles/textes
y = mixed_data_1['label']  # Les labels (1 = Fake)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorisation tfidf (ici probablement des paramètres à fine-tuned, notamment le max_df)

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

#On applique une régression logistique
model_tfidf_logistic = LogisticRegression()
model_tfidf_logistic.fit(X_train_tfidf, y_train)

# Et on évalue notre petit modèle
y_pred = model_tfidf_logistic.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9294144873871789

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93      2075
           1       0.93      0.93      0.93      2246

    accuracy                           0.93      4321
   macro avg       0.93      0.93      0.93      4321
weighted avg       0.93      0.93      0.93      4321


Confusion Matrix:
 [[1920  155]
 [ 150 2096]]


In [29]:
# Appliquer le modèle à fake_real
fake_real_pred = apply_model_tfidf_logistic(fake_real)

print("\nÉvaluation sur Fake real :\n")
print("Accuracy:", accuracy_score(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_real_pred['label'], fake_real_pred['prediction']))


Évaluation sur Fake real :

Accuracy: 0.701026045777427

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.46      0.60      3171
           1       0.63      0.95      0.76      3164

    accuracy                           0.70      6335
   macro avg       0.77      0.70      0.68      6335
weighted avg       0.77      0.70      0.68      6335


Confusion Matrix:
 [[1444 1727]
 [ 167 2997]]


In [30]:
# Appliquer le modèle à mixed_data_2
mixed_data_2_pred = apply_model_tfidf_logistic(mixed_data_2)

print("\nÉvaluation sur new_data_2_pred :\n")
print("Accuracy:", accuracy_score(mixed_data_2_pred['label'], mixed_data_2_pred['prediction']))
print("\nClassification Report:\n", classification_report(mixed_data_2_pred['label'], mixed_data_2_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(mixed_data_2_pred['label'], mixed_data_2_pred['prediction']))


Évaluation sur new_data_2_pred :

Accuracy: 0.9271464938671604

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.92      0.92     10526
           1       0.92      0.94      0.93     11079

    accuracy                           0.93     21605
   macro avg       0.93      0.93      0.93     21605
weighted avg       0.93      0.93      0.93     21605


Confusion Matrix:
 [[ 9638   888]
 [  686 10393]]


In [31]:
# Notre modèle sur mixed_data_2

# On réalise le split des données

X = mixed_data_2['text']  # Les articles/textes
y = mixed_data_2['label']  # Les labels (1 = Fake)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorisation tfidf (ici probablement des paramètres à fine-tuned, notamment le max_df)

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

#On applique une régression logistique
model_tfidf_logistic = LogisticRegression()
model_tfidf_logistic.fit(X_train_tfidf, y_train)

# Et on évalue notre petit modèle
y_pred = model_tfidf_logistic.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9271002082851192

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.91      0.92      2088
           1       0.92      0.94      0.93      2233

    accuracy                           0.93      4321
   macro avg       0.93      0.93      0.93      4321
weighted avg       0.93      0.93      0.93      4321


Confusion Matrix:
 [[1906  182]
 [ 133 2100]]


In [32]:
# Appliquer le modèle à fake_real
fake_real_pred = apply_model_tfidf_logistic(fake_real)

print("\nÉvaluation sur Fake real :\n")
print("Accuracy:", accuracy_score(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_real_pred['label'], fake_real_pred['prediction']))


Évaluation sur Fake real :

Accuracy: 0.690765588003157

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.43      0.58      3171
           1       0.63      0.95      0.75      3164

    accuracy                           0.69      6335
   macro avg       0.76      0.69      0.67      6335
weighted avg       0.76      0.69      0.67      6335


Confusion Matrix:
 [[1366 1805]
 [ 154 3010]]


In [33]:
# Appliquer le modèle à mixed_data_1
mixed_data_1_pred = apply_model_tfidf_logistic(mixed_data_1)

print("\nÉvaluation sur new_data_2_pred :\n")
print("Accuracy:", accuracy_score(mixed_data_1_pred['label'], mixed_data_1_pred['prediction']))
print("\nClassification Report:\n", classification_report(mixed_data_1_pred['label'], mixed_data_1_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(mixed_data_1_pred['label'], mixed_data_1_pred['prediction']))


Évaluation sur new_data_2_pred :

Accuracy: 0.9311733395047442

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93     10526
           1       0.92      0.94      0.93     11079

    accuracy                           0.93     21605
   macro avg       0.93      0.93      0.93     21605
weighted avg       0.93      0.93      0.93     21605


Confusion Matrix:
 [[ 9678   848]
 [  639 10440]]
